<a href="https://colab.research.google.com/github/firmanapriandi/Praktek_Git_SQL/blob/master/SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Library **PostgreSQL**

In [1]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Import Library**

In [2]:
import psycopg2
import pandas as pd

# **Connect Ke Database**

In [6]:
#@title
conn = psycopg2.connect(
    host ="*******",
    database ='*******',
    user = '********',
    password = '********')
conn.set_client_encoding('UTF8')

# **Read Table**

In [9]:
sql1 = """SELECT * FROM public.user_tab;"""
table_user = pd.read_sql_query(sql1, conn)
table_user

,shopid,buyerid,register_date,country
0,2087,4678,2022-07-09,SZ
1,2830,2693,2019-03-05,SG
2,2812,3421,2021-02-10,MY
3,2532,3668,2020-10-05,MY
4,2890,4073,2019-03-06,PH
...,...,...,...,...
10315,2557,3677,2020-10-10,MY
10316,2328,1772,2021-03-11,ID
10317,2996,1019,2018-06-01,ID
10318,2247,4483,2021-08-05,PH


In [15]:
sql2 = """SELECT * FROM public.performance_tab;"""
table_performance = pd.read_sql_query(sql2, conn)
table_performance

,shopid,total_clicks,impressions,item_views
0,2624,3,279,120
1,2708,23,294,182
2,2397,71,218,130
3,2438,3,157,170
4,2991,17,180,165
...,...,...,...,...
10003,2667,56,319,142
10004,2667,56,319,142
10005,2482,71,244,144
10006,2902,57,150,165


In [16]:
sql3 = """SELECT * FROM public.order_tab;"""
table_order = pd.read_sql_query(sql3, conn)
table_order

,shopid,order_id,buyerid,itemid,gmv,order_time
0,2667,1030132,1024,3366770,270.0,2021-07-04
1,2624,1030137,4678,6130641,690.0,2021-12-06
2,2708,1030147,2693,6770063,870.0,2021-04-12
3,2397,1030153,3421,4193426,820.0,2021-04-03
4,2438,1030155,3668,8825994,290.0,2020-02-04
...,...,...,...,...,...,...
18681,2550,1077372,1451,3945750,530.0,2021-10-03
18682,2823,1077373,2380,8167210,360.0,2021-09-09
18683,2491,1077377,3783,1859299,1000.0,2021-08-12
18684,2787,1077381,3428,4329777,930.0,2021-09-05



## **SQL Statement untuk menemukan jumlah buyer per country**



In [14]:
sql4 = """SELECT country, COUNT(buyerid) as total_buyer 
          FROM public.user_tab GROUP BY country;"""
buyer_per_country = pd.read_sql_query(sql4, conn)
buyer_per_country

,country,total_buyer
0,MY,2555
1,ID,2619
2,PH,1275
3,SG,2589
4,SZ,1282


## SQL Statement untuk menemukan jumlah item views untuk setiap shop

In [19]:
sql5 = """SELECT t1.shopid, COUNT(t2.item_views) as total_views
          FROM public.user_tab t1 INNER JOIN public.performance_tab t2
          ON t1.shopid = t2.shopid GROUP BY t1.shopid;"""
item_views_shop = pd.read_sql_query(sql5, conn)
item_views_shop

,shopid,total_views
0,2594,77
1,2571,56
2,2880,180
3,2585,110
4,2806,104
...,...,...
996,2901,35
997,2443,42
998,2539,88
999,2679,156


## SQL Statement untuk menemukan berapa banyak jumlah order yang diterima oleh shop dalam interval 1 bulan

In [22]:
sql6 = """SELECT shopid, COUNT(order_id) as total_order, DATE_TRUNC('month', order_time) as order_month
          FROM public.order_tab GROUP BY shopid, DATE_TRUNC('month', order_time)
          ORDER BY total_order DESC, order_month, shopid ASC;"""
jumlah_order_bulanan = pd.read_sql_query(sql6, conn)
jumlah_order_bulanan

,shopid,total_order,order_month
0,2738,8,2020-04-01 00:00:00+00:00
1,2626,8,2020-07-01 00:00:00+00:00
2,2564,8,2020-08-01 00:00:00+00:00
3,2110,8,2020-10-01 00:00:00+00:00
4,2224,8,2020-10-01 00:00:00+00:00
...,...,...,...
8570,2857,1,2022-07-01 00:00:00+00:00
8571,2878,1,2022-07-01 00:00:00+00:00
8572,2881,1,2022-07-01 00:00:00+00:00
8573,2906,1,2022-07-01 00:00:00+00:00


## SQL Statement untuk menemukan TOP 10 Buyer by GMV dari setiap coutry

In [24]:
sql7 = """SELECT t2.buyerid, COUNT(t2.gmv) as gmv, t1.country
          FROM public.user_tab t1 INNER JOIN public.order_tab t2 ON t1.buyerid = t2.buyerid
          GROUP BY t2.buyerid, t1.country
          ORDER BY gmv DESC LIMIT 10 ;"""
top_10_buyer = pd.read_sql_query(sql7, conn)
top_10_buyer

,buyerid,gmv,country
0,1655,231,ID
1,1895,170,ID
2,4429,153,PH
3,2103,153,SG
4,1415,144,ID
5,1148,135,ID
6,2238,128,SG
7,2126,128,SG
8,1965,120,ID
9,1927,120,ID


## SQL Statement untuk menemukan latet order date dari setiap buyer

In [28]:
sql8 = """SELECT buyerid, order_time FROM order_tab
          GROUP BY order_time, buyerid ORDER BY order_time DESC, buyerid ASC;"""
top_10_buyer = pd.read_sql_query(sql8, conn)
top_10_buyer

,buyerid,order_time
0,1344,2022-07-12
1,1353,2022-07-12
2,1580,2022-07-12
3,1618,2022-07-12
4,1622,2022-07-12
...,...,...
10068,4052,2020-01-01
10069,4146,2020-01-01
10070,4343,2020-01-01
10071,4398,2020-01-01
